In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
# !sudo pip install 'prompt-toolkit==1.0.15'
# !pip install simpletransformers 
# !pip install checklist
# !pip install spacy
# !pip install -U ipython 

In [8]:
import numpy as np
import pandas as pd
import spacy
from collections import Counter
#
from simpletransformers.classification import ClassificationModel
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
#
np.random.seed(142)
editor = Editor()
nlp = spacy.load('en_core_web_sm')

In [9]:
train = pd.read_csv('/content/gdrive/MyDrive/olid-train.csv', index_col=0)
test = pd.read_csv('/content/gdrive/MyDrive/olid-test.csv', index_col=0)

In [10]:
!unzip gdrive/My\Drive/outputs.zip

Archive:  gdrive/MyDrive/outputs.zip
Written using ZipTricks 5.6.0
 extracting: outputs/config.json     
 extracting: outputs/eval_results.txt  
 extracting: outputs/model_args.json  
 extracting: outputs/pytorch_model.bin  
 extracting: outputs/special_tokens_map.json  
 extracting: outputs/tokenizer.json  
 extracting: outputs/tokenizer_config.json  
 extracting: outputs/training_args.bin  
 extracting: outputs/vocab.txt       


In [11]:
model = ClassificationModel('bert', 'outputs', use_cuda=False)


In [13]:
tweets = list(train['text'])
#tweets
#for tweet in tweets:
#  for word in tweet.split():
#    if word[:2]=='##':
#      print('exists')

Tokenizer splits tokens in subwords using ##


In [58]:
#4a
count_t =  Counter()
n_splitted = 0

for tweet in tweets:
  tokened = model.tokenizer.tokenize(tweet)
  tokens = [i for i in tokened if (len(i)>2 and i[:2]!='##') or len(i)<=2] #selects tokens
  count_t.update(tokens)
  for j in range(len(tokened)-1):
    if tokened[j] in tokens and tokened[j+1] not in tokens: #if jth is token and j+1 th is not then splitted 
      n_splitted +=1

numberoftokens = sum(count_t.values())
print(numberoftokens, 'number of tokens')
print(n_splitted, 'number of splitted tokens')


387931 number of tokens
67045 number of splitted tokens


In [ ]:
#b
s = 0
for tweet in tweets: #subword of a not splitted word is equal to 1
  tokenedtrain = model.tokenizer.tokenize(tweet)
  s+= len(tokenedtrain)
av_sub = s/numberoftokens; av_sub

1.2346396653013036

In [59]:
#c 
n = 0
for word in tokened:
    if n==10:
      #print(tokened)
      break
    if len(word)>2 and word[:2]=='##':
      if previous in count_t.keys():
        n+=1
        print(previous, word)
        previous = word
      else:
        print(word)
    else:
        previous = word
    

Spanish ##re
##venge
Human ##R
##ight
##s
Freedom ##O
##f
##E
##x
##press
##ion
fake ##de
##mo
##cracy
US ##ER
US ##ER
US ##ER
US ##ER
US ##ER
US ##ER


In [60]:
#d
vocab = list(model.tokenizer.vocab)
counts = [(len(i), vocab.index(i)) for i in vocab if len(i)>2 and i[:2]=='##'] #(count, vocab.index)
Longest = max(counts)
(Longest, vocab[counts[counts.index(Longest)][1]])
#max([(1,20), (199,1)])

((16, 21904), '##sunderstanding')

In [63]:
print([i for i in vocab if len(i)>=16 and i[:2]=='##'])

['##sunderstanding']


In [ ]:
len(vocab)

28996

In [ ]:
##7
ret_nonoff = editor.template("I don't hate {mask}")
ret_off = editor.template("I hate {mask}")
#
pred_off = model.predict(ret_nonoff.data)
pred_nonoff = model.predict(ret_off.data)

print(ret_nonoff.data)
print(pred_nonoff[0])
print(ret_off.data)
print(pred_off[0])

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

["I don't hate them", "I don't hate you", "I don't hate him", "I don't hate people", "I don't hate her", "I don't hate Trump", "I don't hate anyone", "I don't hate myself", "I don't hate it", "I don't hate God", "I don't hate anybody", "I don't hate Muslims", "I don't hate Jews", "I don't hate gays", "I don't hate religion", "I don't hate women", "I don't hate racism", "I don't hate anything", "I don't hate politics", "I don't hate Hitler", "I don't hate men", "I don't hate me", "I don't hate .", "I don't hate Nazis", "I don't hate violence", "I don't hate that", "I don't hate death", "I don't hate Obama", "I don't hate capitalism", "I don't hate hate", "I don't hate ISIS", "I don't hate war", "I don't hate immigrants", "I don't hate America", "I don't hate children", "I don't hate Putin", "I don't hate everyone", "I don't hate Russians", "I don't hate Israel", "I don't hate […]", "I don't hate love", "I don't hate others", "I don't hate …", "I don't hate kids", "I don't hate Islam", "

In [67]:
religion_nonoff = editor.template("I don't hate {religion}")
religion_off = editor.template("I hate {religion}")
rel_off_predictions=model.predict(religion_off.data)
rel_nonoff_predictions=model.predict(religion_nonoff.data)

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [68]:
print(religion_off.data[0:10])
print(rel_off_predictions[0])
print(religion_nonoff.data[0:10])
print(rel_nonoff_predictions[0])

['I hate Christianity', 'I hate Judaism', 'I hate Islam', 'I hate Buddhism', 'I hate Zoroastrianism', 'I hate Hinduism', 'I hate Sikhism', 'I hate Shintoism', "I hate Baha'i", 'I hate Taoism']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
["I don't hate Christianity", "I don't hate Judaism", "I don't hate Islam", "I don't hate Buddhism", "I don't hate Zoroastrianism", "I don't hate Hinduism", "I don't hate Sikhism", "I don't hate Shintoism", "I don't hate Baha'i", "I don't hate Taoism"]
[0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]


In [69]:
nat_nonoff = editor.template("I don't hate {nationality}")
nat_off = editor.template("I hate {nationality}")

In [70]:
nat_off_predictions=model.predict(nat_off.data)
nat_nonoff_predictions=model.predict(nat_nonoff.data)

  0%|          | 0/215 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/215 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

In [71]:
print(nat_off.data[0:10])
print(nat_off_predictions[0])
print(nat_nonoff.data[0:10])
print(nat_nonoff_predictions[0])

['I hate Chinese', 'I hate Indian', 'I hate American', 'I hate Indonesian', 'I hate Pakistani', 'I hate Brazilian', 'I hate Nigerian', 'I hate Bangladeshi', 'I hate Russian', 'I hate Japanese']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
["I don't hate Chinese", "I don't hate Indian", "I don't hate American", "I don't hate Indonesian", "I don't hate Pakistani", "I don't hate Brazilian", "I don't